In [5]:
api_key='MyjBpIrnMbCmsxXJD6zmiOCWqjB3p+7Icq5PRSrN82IXeOppkoEWybx1SmRryxT+'

In [10]:
%run import_modules.ipynb

In [11]:
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def extract_team_data(year, week):
    """
    Extracts team statistics and converts them directly into a DataFrame.
    
    :param year: Year of the game data.
    :param week: Week of the game data.
    :return: DataFrame containing extracted team statistics for each game.
    """
    
    # Helper function to process a single game
    def process_game(game):
        game_id = game.id
        teams_data = []
        
        for team in game.teams:
            stats_dict = {stat.category: stat.stat for stat in team.stats}
            teams_data.append({
                'game_id': game_id,
                'team_id': team.school_id,
                'team_name': team.school,
                'home_away': team.home_away,
                'points': team.points,
                **stats_dict
            })
        
        return teams_data

    # Retrieve game data from the API
    game_data = api_instance.get_team_game_stats(year=year, week=week)

    # Use parallel processing to handle the games
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(process_game, game_data))
    
    # Flatten the list of lists
    flattened_results = [item for sublist in results for item in sublist]
    
    # Create DataFrame directly from the processed data
    df = pd.DataFrame(flattened_results).fillna(0)  # Replace NaNs with 0 if needed

    return df

In [12]:
extract_team_data(year=2023,week=1)

,game_id,team_id,team_name,home_away,points,rushingTDs,passingTDs,kickReturnYards,kickReturnTDs,kickReturns,...,firstDowns,puntReturnYards,puntReturnTDs,puntReturns,tacklesForLoss,defensiveTDs,tackles,sacks,qbHurries,passesDeflected
0,401540244,2320,Lamar,home,17,1,1,82,0,4,...,16,0,0,0,0,0,0,0,0,0
1,401540244,70,Idaho,away,42,2,4,0,0,1,...,20,26,0,3,0,0,0,0,0,0
2,401525434,2426,Navy,away,3,0,0,12,0,1,...,12,0,0,0,2,0,42,0,1,2
3,401525434,87,Notre Dame,home,42,2,4,41,0,2,...,27,11,0,1,5,0,41,2,2,0
4,401525435,2634,Tennessee State,away,3,0,0,76,0,2,...,12,0,0,0,6,0,45,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,401520158,55,Jacksonville State,home,49,6,1,0,0,0,...,25,16,0,2,8,0,25,3,5,1
256,401540354,124386,Arkansas Baptist,away,0,0,0,131,0,6,...,10,-2,0,1,0,0,0,0,0,0
257,401540354,2277,Houston Christian,home,66,5,3,17,0,1,...,20,49,0,4,0,0,0,0,0,0
258,401520148,158,Nebraska,away,10,0,1,78,0,2,...,18,8,0,2,5,0,36,3,3,4
